In [3]:
from Bsor import make_bsor, Bsor, save_replay_to_file
from AccuracyCalculator import AccuracyCalculator
from interpretMapFiles import create_map, Map

from Saber import Saber
from NoteCutter import NoteCutter
from noteManager import NoteManager

import requests
import json
import os

def recalculate_replay(map_data: Map, replay: Bsor) -> int:
    left_saber = Saber(0)
    right_saber = Saber(1)

    note_cutter = NoteCutter()
    note_manager = NoteManager(map_data, replay)
    accuracy_calculator = AccuracyCalculator()

    print("# frames:", len(replay.frames))

    count = 0

    frames = replay.frames

    for frame in frames:
        if count > 0:
            note_cutter.cut(left_saber, note_manager, replay)
            note_cutter.cut(right_saber, note_manager, replay)

        left_saber.manual_update(frame.left_hand, frame.time)
        right_saber.manual_update(frame.right_hand, frame.time)

        note_manager.update(frame)

        count += 1
        if count % 1000 == 0:
            print("Processed Frame:", count)

    events = note_manager.get_events()
    counter = 0

    for real in replay.notes:
        for predicted in events:
            if predicted.note_id == real.note_id and abs(predicted.spawn_time - real.spawn_time) < 0.01:
                if predicted.event_type != real.event_type or predicted.event_time != real.event_time:
                    # print(f"Predicted: {predicted.score.value}, Real: {real.score.value}. {predicted.cut.cutPoint} {real.cut.cutPoint}")
                    print(f"Time: {real.event_time} -> {predicted.event_time} Event: {real.event_type} -> {predicted.event_type}")
                    counter += 1
                elif hasattr(predicted, 'cut') and hasattr(real, 'cut'):
                    if round(predicted.cut.cutDistanceToCenter, 2) != round(real.cut.cutDistanceToCenter, 2) or round(predicted.cut.beforeCutRating, 2) != round(real.cut.beforeCutRating, 2):
                        counter += 1
                        print(f"Acc: {real.cut.cutDistanceToCenter:.2f} -> {predicted.cut.cutDistanceToCenter:.2f} Pre: {real.cut.beforeCutRating:.2f} -> {predicted.cut.beforeCutRating:.2f} After: {real.cut.afterCutRating:.2f} -> {predicted.cut.afterCutRating:.2f} ")
                    elif round(predicted.cut.cutPoint[2], 2) != round(real.cut.cutPoint[2], 2):
                        counter += 1
                        print(f"Cut point: {real.cut.cutPoint[2]:.2f} -> {predicted.cut.cutPoint[2]:.2f}")
                break
    
    print(f"Wrong events: {counter} / {len(replay.notes)}")
    replay.notes = events

    return accuracy_calculator.calculate_accuracy(replay)

MAPS_PATH = '.\\maps\\'
REPLAY_PATH = '.\\replays\\5962011.bsor'

with open(REPLAY_PATH, 'rb') as f:
    replay = make_bsor(f)
    f.close()

# replay.info.playerId = '19573' # PP bot ID

map_hash = replay.info.songHash

# Check if map is already downloaded
map_path = MAPS_PATH + map_hash
if not os.path.isfile(map_path):
    # Download map zip from BeatSaver
    import zipfile
    import io
    import requests

    url = f"https://beatsaver.com/api/maps/hash/{map_hash}"
    data = json.loads(requests.get(url).text)
    map_link = data["versions"][0]["downloadURL"]

    r = requests.get(map_link)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(map_path)

# replay.info.playerId = '19573' # PP bot ID

mapFile = create_map(map_path)

print("Calculated score:", recalculate_replay(mapFile, replay))
print("Actual score:", replay.info.score)

save_replay_to_file(replay, ".\\generatedReplays\\generatedwithscore.bsor")

# frames: 12178
Processed Frame: 1000
Processed Frame: 2000
Processed Frame: 3000
Processed Frame: 4000
Processed Frame: 5000
Processed Frame: 6000
Processed Frame: 7000
Processed Frame: 8000
Processed Frame: 9000
Processed Frame: 10000
Processed Frame: 11000
Processed Frame: 12000
Acc: 0.29 -> 0.29 Pre: 1.73 -> 1.14 After: 1.94 -> 2.62 
Acc: 0.17 -> 0.17 Pre: 1.00 -> 1.49 After: 1.78 -> 2.03 
Acc: 0.35 -> 0.35 Pre: 1.00 -> 1.73 After: 1.58 -> 1.63 
Acc: 0.38 -> 0.38 Pre: 1.55 -> 1.02 After: 1.67 -> 2.90 
Acc: 0.24 -> 0.24 Pre: 1.00 -> 1.34 After: 1.52 -> 2.37 
Acc: 0.45 -> 0.45 Pre: 1.00 -> 1.55 After: 1.54 -> 2.02 
Acc: 0.08 -> 0.08 Pre: 1.58 -> 1.05 After: 1.64 -> 3.00 
Acc: 0.18 -> 0.18 Pre: 1.00 -> 1.42 After: 1.39 -> 2.39 
Acc: 0.18 -> 0.18 Pre: 1.00 -> 1.58 After: 1.48 -> 2.12 
Acc: 0.04 -> 0.04 Pre: 1.00 -> 1.27 After: 1.61 -> 1.81 
Acc: 0.35 -> 0.35 Pre: 1.00 -> 1.49 After: 1.38 -> 1.45 
Acc: 0.35 -> 0.35 Pre: 1.83 -> 1.18 After: 1.71 -> 2.49 
Acc: 0.52 -> 0.52 Pre: 1.00 -> 1.

In [4]:
from Bsor import make_bsor, Bsor, save_replay_to_file
from AccuracyCalculator import AccuracyCalculator
from interpretMapFiles import create_map, Map

from Saber import Saber
from NoteCutter import NoteCutter
from noteManager import NoteManager

import requests
import json
import urllib.request
import os

def recalculate_replay(map_data: Map, replay: Bsor) -> int:
    left_saber = Saber(0)
    right_saber = Saber(1)

    note_cutter = NoteCutter()
    note_manager = NoteManager(map_data, replay)
    accuracy_calculator = AccuracyCalculator()
    count = 0

    frames = replay.frames

    for frame in frames:
        if count > 0:
            note_cutter.cut(left_saber, note_manager, replay)
            note_cutter.cut(right_saber, note_manager, replay)

        left_saber.manual_update(frame.left_hand, frame.time)
        right_saber.manual_update(frame.right_hand, frame.time)

        note_manager.update(frame)

        count += 1
        print(f"{count} / {len(frames)}", end='\r')
            

    events = note_manager.get_events()
    replay.notes = events

    return accuracy_calculator.calculate_accuracy(replay)

MAPS_PATH = '.\\maps\\'
SCORE_ID = 6408746

# Download file from API with Score ID
url = f"https://api.beatleader.xyz/score/{SCORE_ID}"
data = json.loads(requests.get(url).text)
replay_link = data["replay"]

# Download replay file from link
replay_path = '.\\replays\\' + str(SCORE_ID) + '.bsor'
if not os.path.isfile(replay_path):
    # add headers to download file
    opener = urllib.request.build_opener()
    opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
    urllib.request.install_opener(opener)
    urllib.request.urlretrieve(replay_link, replay_path)

with open(replay_path, 'rb') as f:
    replay = make_bsor(f)
    f.close()

map_hash = replay.info.songHash

# Check if map is already downloaded
map_path = MAPS_PATH + map_hash
if not os.path.isfile(map_path):
    # Download map zip from BeatSaver
    import zipfile
    import io
    import requests

    url = f"https://beatsaver.com/api/maps/hash/{map_hash}"
    data = json.loads(requests.get(url).text)
    map_link = data["versions"][0]["downloadURL"]

    r = requests.get(map_link)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(map_path)

# replay.info.playerId = '19573' # PP bot ID

mapFile = create_map(map_path)

print("Calculated score:", recalculate_replay(mapFile, replay))
print("Actual score:", replay.info.score)

save_replay_to_file(replay, ".\\generatedReplays\\generatedwithscore.bsor")

NOT OK <356.6058727034155, 340.4544750883613, 134.76469084172217>
NOT OK -29.149468785885556 57.361083984375
NOT OK <5.5892534151473345, 348.75551323613354, 315.0197314595552>
NOT OK 164.14042287695918 136.66650390625
Calculated score: 1074133.0
Actual score: 1074141
